In [2]:
!pip3 install pandas
!pip3 install yfinance
!pip3 install scikit-learn
!pip3 install xgboost
!pip3 install mwclient
!pip3 install transformers
!pip install textblob
!pip install torch
!pip install tensorflow


In [3]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [4]:
revs = list(page.revisions())

In [5]:
revs[0]

OrderedDict([('revid', 1223706292),
             ('parentid', 1223701129),
             ('user', 'Grayfell'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=5, tm_mday=13, tm_hour=20, tm_min=49, tm_sec=27, tm_wday=0, tm_yday=134, tm_isdst=-1)),
             ('comment',
              "Undid revision [[Special:Diff/1223701129|1223701129]] by [[Special:Contributions/Gjb0zWxOb|Gjb0zWxOb]] ([[User talk:Gjb0zWxOb|talk]]) That is definitely not one of the key points of that source. The source doesn't specifically mention Lightning, either.")])

In [6]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [7]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [9]:
from textblob import TextBlob

def find_sentiment(text):
    testimonial = TextBlob(text)
    return testimonial.sentiment.polarity


In [10]:
from textblob import TextBlob

edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    sentiment = TextBlob(comment).sentiment.polarity
    edits[date]["sentiments"].append(sentiment)


In [11]:
from statistics import mean
for key in edits:
    sentiments = edits[key].get("sentiments", [])
    num_sentiments = len(sentiments)
    
    avg_sentiment = mean(sentiments) if num_sentiments > 0 else 0
    neg_sentiment_ratio = len([s for s in sentiments if s < 0]) / num_sentiments if num_sentiments > 0 else 0
    
    edits[key] = {
        "sentiment": avg_sentiment,
        "neg_sentiment": neg_sentiment_ratio,
        "edit_count": edits[key]["edit_count"]
    }


In [19]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [20]:
edits_df

,sentiment,neg_sentiment,edit_count
2009-03-08,0.000000,0.000000,4
2009-08-05,0.000000,0.000000,1
2009-08-06,0.000000,0.000000,2
2009-08-14,0.000000,0.000000,1
2009-10-13,0.000000,0.000000,2
...,...,...,...
2024-04-27,-0.200000,0.666667,3
2024-04-30,0.034091,0.000000,4
2024-05-04,0.000000,0.000000,2
2024-05-05,0.066667,0.000000,3


In [23]:
edits_df.index = pd.to_datetime(edits_df.index)

In [41]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [27]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [31]:
edits_df

,sentiment,neg_sentiment,edit_count
2009-03-08,0.0,0.0,4
2009-03-09,0.0,0.0,0
2009-03-10,0.0,0.0,0
2009-03-11,0.0,0.0,0
2009-03-12,0.0,0.0,0
...,...,...,...
2024-05-11,0.0,0.0,0
2024-05-12,0.0,0.0,0
2024-05-13,0.0,0.0,2
2024-05-14,0.0,0.0,0


In [33]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [35]:
rolling_edits = rolling_edits.dropna()

In [37]:
rolling_edits

,sentiment,neg_sentiment,edit_count
2009-04-06,0.000000,0.000000,0.133333
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-05-11,-0.005429,0.055556,0.500000
2024-05-12,-0.005429,0.055556,0.500000
2024-05-13,-0.005429,0.055556,0.566667
2024-05-14,-0.005429,0.055556,0.566667


In [39]:
rolling_edits.to_csv("wikipedia_edits.csv")